In [33]:
# Carga paquetes
import pandas as pd
import numpy as np
import warnings
from scipy.stats import skew

warnings.filterwarnings("ignore", category=UserWarning)  # ignora los warnings

In [34]:
###########    FUNCIONES   ##########
## Interquartile Range [IQR)
#Se calculan los limites superiores e inferiores para identificación de posibles datos atipicos
# IQR = Q3 - Q1 
# LI = Q1–1.5 * IQR; LS = Q3+1.5 * IQR.
def IQR(variable):
  Iqr = np.percentile(variable,75)-np.percentile(variable,25)

  LI=np.percentile(variable,25)-(1.5*Iqr)
  LS=np.percentile(variable,75)+(1.5*Iqr)
  return(LI,LS)

In [41]:
def categorizacion (variable,outliers,reciencia):
  LI = outliers[0]
  LS = outliers[1]

  # variable tiene outliers, VarTemp no
  VarTemp = variable[(variable >= LI) & (variable <= LS)]

  ## TIPO DE INTERVALO O BREAK  
  # valores en el que se dividen los intervalos. 
  # los datos atipicos se apartan a la hora de definir los quantiles, pero se incluyen al final para el puntaje
  # break_1: <LI + min; percentiles -1 y 6; max + >LS
  breaks_1 = np.concatenate(([min(variable) - 0.001]\
    ,np.percentile(VarTemp, np.arange(0, 101, 20)[1:5]) #np.arange(start, stop, step), stop excluded so +1/
    ,[max(variable) + 0.001]))

  # break_2: quantiles con outliers
  quantiles_2 = np.percentile(variable, np.arange(0, 101, 20))
  quantiles_2[0] -= 0.001
  quantiles_2[-1] += 0.001
  breaks_2 = quantiles_2
 
  # break_3: Usandos rangos fijos y no quantiles incluyendo outliers
  breaks_3 = np.linspace(min(variable) - 0.001, max(VarTemp), num=5)
  breaks_3 = np.append(breaks_3, max(variable))

  if not reciencia:
    # Usar el break sin outliers sin que quantiles repitan valores
    if not np.any(pd.Series(breaks_1).duplicated()):
        puntaje = pd.cut(variable, bins=breaks_1, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_1)
    # Usar el break con outliers sin que quantiles repitan valores
    elif np.any(pd.Series(breaks_1).duplicated()) and not np.any(pd.Series(breaks_2).duplicated()):
        puntaje = pd.cut(variable, bins=breaks_2, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_2)
    # Usar el break con outliers sin usar quantiles sino rangos fijos    
    else:
        puntaje = pd.cut(variable, bins=breaks_3, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_3)

# restar 6 y multiplicar -1 invierte el puntaje para Recencia, es decir, si el puntaje mayor es 5 para los valores mas grandes 
# en los otros casos, en Recencia los menores son los de mayor puntaje
  else:
    if not np.any(pd.Series(breaks_1).duplicated()):
        puntaje = ((pd.cut(variable, bins=breaks_1, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_1)
    elif np.any(pd.Series(breaks_1).duplicated()) and not np.any(pd.Series(breaks_2).duplicated()):
        puntaje = ((pd.cut(variable, bins=breaks_2, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_2)
    else:
        puntaje = ((pd.cut(variable, bins=breaks_3, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_3)

  return (puntaje,categorias)

In [36]:
#Función para el caso WHEN
def case_when(x):
    if x in[17,18,19,20]:
        return 'Muy Alto'
    elif x in[13,14,15,16]:
        return 'Alto'
    elif x in[8,9,10,11,12]:
        return 'Medio'
    else:
        return 'Bajo'

In [45]:
def puntajes_RFM (tb):
  rfm_df = tb
  
  var_eventos = rfm_df['Eventos']
  var_recaudo = rfm_df['Recaudo']
  var_boletas = rfm_df['Boletas']
  var_atp = rfm_df['ATP']

  # Comprobamos normalidad, sino, transformamos con log para reducir
  # sesgo a la derecha:
  #Ejecutamos las dos funciones anteriores
  skew_lim = 2
  
  # Eventos
  if skew(var_eventos) > skew_lim:
    # Aplicar logaritmo a var_eventos si la asimetría es mayor que skew
    var_eventos = np.log(var_eventos)
    rfm_df.loc[:, 'Eventos_t'] = var_eventos # esta sintaxis evita el warning 'SettingWithCopyWarning'
    #rfm_df['Eventos_t'] = var_eventos  # Asignar var_eventos a una columna Eventos_t en rfm_df
    lim_eventos = IQR(var_eventos)  # Calcular el rango intercuartílico de var_eventos
  else:
    lim_eventos = IQR(var_eventos)
  
  # Recaudo
  if skew(var_recaudo) > skew_lim:
    # Aplicar logaritmo a var_eventos si la asimetría es mayor que skew
    var_recaudo = np.log(var_recaudo)
    #rfm_df['Recaudo_t'] = var_recaudo  # Asignar var_eventos a una columna Eventos_t en rfm_df
    rfm_df.loc[:, 'Recaudo_t'] = var_recaudo
    lim_recaudo = IQR(var_recaudo)  # Calcular el rango intercuartílico de var_eventos
  else:
    lim_recaudo = IQR(var_recaudo)
  
  # Boletas
  if skew(var_boletas) > skew_lim:
    # Aplicar logaritmo a var_eventos si la asimetría es mayor que skew
    var_boletas = np.log(var_boletas)
    rfm_df.loc[:, 'Recaudo_t'] = var_recaudo
    #rfm_df['Boletas_t'] = var_boletas  # Asignar var_eventos a una columna Eventos_t en rfm_df
    lim_boletas = IQR(var_boletas)  # Calcular el rango intercuartílico de var_eventos
  else:
    lim_boletas = IQR(var_boletas)
  
  # ATP
  if skew(var_atp) > skew_lim:
    # Aplicar logaritmo a var_eventos si la asimetría es mayor que skew
    var_atp = np.log(var_atp)
    rfm_df.loc[:, 'ATP_t'] = var_atp
    #rfm_df['ATP_t'] = var_atp  # Asignar var_eventos a una columna Eventos_t en rfm_df
    lim_ATP = IQR(var_atp)  # Calcular el rango intercuartílico de var_eventos
  else:
    lim_ATP = IQR(var_atp)

  # Aplicamos la funcion de categorizacion
  cat_eventos = categorizacion(var_eventos,lim_eventos, reciencia= True)
  cat_recaudo = categorizacion(var_recaudo,lim_recaudo, reciencia=False)
  cat_boletas = categorizacion(var_boletas,lim_boletas, reciencia=False)
  cat_atp = categorizacion(var_atp,lim_ATP, reciencia=False)


  # Campos de puntaje
  rfm_df['rango_eventos'] = cat_eventos[1]
  rfm_df['rango_recaudo'] = cat_recaudo[1]
  rfm_df['rango_boletas'] = cat_boletas[1]
  rfm_df['rango_atp'] = cat_atp[1]

  # Campos de rango 
  rfm_df['puntaje_eventos'] = cat_eventos[0]
  rfm_df['puntaje_recaudo'] = cat_recaudo[0]
  rfm_df['puntaje_boletas'] = cat_boletas[0]
  rfm_df['puntaje_atp'] = cat_atp[0]
  
  rfm_df['score'] = rfm_df['puntaje_eventos'] + rfm_df['puntaje_recaudo']\
     + rfm_df['puntaje_boletas'] + rfm_df['puntaje_atp']

  #rfm_df['Score'] = rfm_df['puntaje_recencia'] + rfm_df['puntaje_frecuencia'] + rfm_df['puntaje_monto']
  
  rfm_df['Valor'] = rfm_df['score'].apply(lambda x: case_when(x))

  return(rfm_df)


In [44]:
# Configuración para evitar la notación científica
np.set_printoptions(suppress=True)

# Carga de la base
#inpath = "C:/Users/Diego Torres/OneDrive/Datasets/Manar/"
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/"

tb_ini = pd.read_csv(inpath + 'clientes_2023.csv',delimiter=';',decimal=",")
tb_ini.fillna(0, inplace=True)

#################
# Descriptivos
print(tb_ini.sample(5))
print(' ')
print('Summary:')
tb_ini.info()
print(' ')
print('Descriptivos:')
print(tb_ini.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(tb_ini.isnull().sum())  # total de nulls por variable
print(' ')
print('Dimensiones:')
print(tb_ini.shape)

             Cliente  Eventos   Recaudo  Boletas       ATP
538960  1.022930e+13        1   75600.0        2   37800.0
529213  1.022928e+13        1  100000.0        2   50000.0
464730  1.022915e+13        1   75000.0        6   12500.0
294778  1.022865e+13        1  379000.0        1  379000.0
304020  1.022870e+13        1   90000.0        2   45000.0
 
Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732382 entries, 0 to 732381
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Cliente  732382 non-null  float64
 1   Eventos  732382 non-null  int64  
 2   Recaudo  732382 non-null  float64
 3   Boletas  732382 non-null  int64  
 4   ATP      732382 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 27.9 MB
 
Descriptivos:
            count          mean           std     min           25%  \
Cliente  732382.0  8.902537e+12  3.418647e+12     0.0  1.022844e+13   
Eventos  732382.0  2.230000e+00  5.260000e+

In [39]:
# terminamos de depurar registros con valores 0 en alguna de sus entradas
tb = tb_ini[(tb_ini['Eventos'] > 0) & (tb_ini['Recaudo'] > 0) & (tb_ini['Boletas'] > 0)]
#tb = tb_ini[(tb_ini['Eventos'] == 0) | (tb_ini['Recaudo'] == 0) | (tb_ini['Boletas'] == 0)]
print('Dimensiones:')
print(tb.shape)
print(tb.head(5))

Dimensiones:
(732381, 5)
        Cliente  Eventos    Recaudo  Boletas            ATP
1  1.013229e+11        1    26400.0        2   13200.000000
2  1.013230e+11        3   989000.0        5  197800.000000
3  1.013230e+11        2   654000.0        4  163500.000000
4  1.013264e+11        3   119000.0        6   19833.333333
5  1.013264e+11        4  1343000.0       28   47964.285714


In [46]:
#############################
### TABLA RESULTADO
base_resultado = puntajes_RFM(tb)

print(base_resultado.head(5))
print(' ')
print('Dimensiones:')
print(base_resultado.shape)
print(' ')
print('Summary:')
base_resultado.info()

        Cliente  Eventos    Recaudo  Boletas            ATP  Eventos_t  \
1  1.013229e+11        1    26400.0        2   13200.000000   0.000000   
2  1.013230e+11        3   989000.0        5  197800.000000   1.098612   
3  1.013230e+11        2   654000.0        4  163500.000000   0.693147   
4  1.013264e+11        3   119000.0        6   19833.333333   1.098612   
5  1.013264e+11        4  1343000.0       28   47964.285714   1.386294   

   Recaudo_t  Boletas_t      ATP_t    rango_eventos     rango_recaudo  \
1  10.181119   0.693147   9.487972  (-0.001, 0.402]   (8.388, 11.408]   
2  13.804450   1.609438  12.195012   (0.804, 1.207]   (13.59, 20.847]   
3  13.390863   1.386294  12.004568   (0.402, 0.804]   (12.848, 13.59]   
4  11.686879   1.791759   9.895119   (0.804, 1.207]  (11.408, 12.196]   
5  14.110416   3.332205  10.778212   (1.207, 1.609]   (13.59, 20.847]   

     rango_boletas         rango_atp  puntaje_eventos  puntaje_recaudo  \
1  (-0.001, 0.735]     (7.6, 10.316]      